<a href="https://colab.research.google.com/github/HiteshAI/Plant_Classification/blob/developer/plant_classification_tensorflow_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


tf.enable_eager_execution()


In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/plant_classification/datasets


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_train_valid_data(batch_size, train_dir, IMG_HEIGHT, IMG_WIDTH):
    image_gen_train = ImageDataGenerator(
        rescale=1. / 255,
        validation_split=0.2,
        rotation_range=45,
        width_shift_range=.15,
        height_shift_range=.15,
        horizontal_flip=True,
        zoom_range=0.5
    )

    image_gen_val = ImageDataGenerator(
        rescale=1. / 255,
        validation_split=0.2,
    )

    train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                         directory=train_dir,
                                                         shuffle=True,
                                                         target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                         class_mode='categorical',
                                                         subset='training'
                                                         )

    label_map = (train_data_gen.class_indices)
    print(label_map)

    val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='categorical',
                                                     subset='validation',

                                                     )

    sample_training_images, _ = next(train_data_gen)
    sample_testing_images, _ = next(val_data_gen)
    return train_data_gen, val_data_gen



def plot_acc_and_validation(history, model_name):
    acc = history.history['acc']
    val_acc = history.history['val_acc']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # If early stopping not used add epcos_range to plot graph for complete epochs
    # epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    import os
    my_path = os.path.abspath('/content/drive/My Drive/plant_classification/plot_results')


    my_file = model_name + 'plot.png'
    plt.savefig(os.path.join(my_path, my_file))


def evaluation_summary_prec_f1_score(model, val_data_gen, batch_size):
    validation_steps_per_epoch = (val_data_gen.samples // batch_size)
    predictions = model.predict_generator(val_data_gen, steps=validation_steps_per_epoch)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = val_data_gen.classes
    class_labels = list(val_data_gen.class_indices.keys())

    from sklearn.metrics import classification_report

    report = classification_report(true_classes, predicted_classes, target_names=class_labels)
    print(report)


In [0]:
%cd /content/drive/My Drive/plant_classification
batch_size = 64
epochs = 50
IMG_HEIGHT = 64
IMG_WIDTH = 64
OUTPUT_CLASS = 52

DATASET_PATH = '/content/drive/My Drive/plant_classification/datasets/plant_leaf'
MODEL_PATH = '/content/drive/My Drive/plant_classification/final_model_plant_leaf'
os.makedirs(MODEL_PATH, exist_ok=True)
MODEL_NAME = os.path.join(MODEL_PATH, 'model_' + str(epochs) + '.h5')


checkpoint_path = "training_leaf_classification/cp-{epoch:04d}.ckpt"
keras_callbacks = [
    EarlyStopping(monitor='val_loss', patience=50, mode='auto'),
    # TensorBoard(log_dir='/home/hitesh/Documents/THESIS/tensorflow_thesis_scripts/tensorboard_logs', histogram_freq=0,
    #             write_graph=False),
    ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min')
]




# checkpoint_path = "training_rice_classification/cp-{epoch:04d}.ckpt"
train_dir = os.path.join(DATASET_PATH, 'train')


train_data_gen, val_data_gen = get_train_valid_data(batch_size=batch_size, train_dir=train_dir,
                                                        IMG_HEIGHT=IMG_HEIGHT,
                                                        IMG_WIDTH=IMG_WIDTH)



base_model = VGG16(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(52, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional VGG16 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs

model.load_weights('/content/drive/My Drive/plant_classification/training_leaf_classification/cp-0001.ckpt')


model.fit_generator(
        train_data_gen,
        steps_per_epoch=train_data_gen.samples // batch_size,
        initial_epoch = 2,
        epochs=epochs,
        callbacks=keras_callbacks,
        validation_data=val_data_gen,
        validation_steps=val_data_gen.samples // batch_size
    )




# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:10]:
   layer.trainable = False
for layer in model.layers[10:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)



# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit_generator(
        train_data_gen,
        steps_per_epoch=train_data_gen.samples // batch_size,
        epochs=epochs,
        callbacks=keras_callbacks,
        validation_data=val_data_gen,
        validation_steps=val_data_gen.samples // batch_size,

    )

plot_acc_and_validation(history, 'model_plant')
model.save(MODEL_NAME + 'model_plant')
model1 = tf.keras.models.load_model(MODEL_NAME + 'model_plant')
evaluation_summary_prec_f1_score(model1, val_data_gen, batch_size)


For Persistent Training